# Youtube URL study

## Import

In [1]:
import pandas as pd

from matplotlib.pyplot import figure
import matplotlib.pyplot as plt

import sys
import url_util as ut
sys.path.insert(0, '../')
import general_utils as gen_ut

## Dataset preparation

In [2]:
df_yt_complete = pd.read_csv('Youtube_URL.csv',low_memory=False)
df_yt_complete = df_yt_complete.groupby('url').first().sort_values(by='occurrency',ascending=False)
df_yt_complete.head()

,title,description,type,available,reason,occurrency
url,,,,,,
https://www.youtube.com/watch?v=U-kC9XkT7Eo,I LOCKDOWNS: DISTRUGGERANNO PIU' VITE DI QUANT...,Intervista all'epidemiologa di punta della Har...,decompressed,True,Parsed,508
https://www.youtube.com/watch?v=xgfQfDh4puk,"DAL TOP DI HARVARD, LE INCOGNITE DI CoV-2, IMM...",David R. Walt della Harvard University Medical...,decompressed,True,Parsed,346
https://www.youtube.com/watch?v=-BbnIlxaqJM,VACCINI OBBLIGATORI AGGRAVEREBBERO LA SITUAZIO...,ISCRIVITI AL NOSTRO CANALE YOUTUBE: https://bi...,decompressed,True,Parsed,265
https://youtu.be/kHGtn_vnrJ8,IL PARERE DEL PREMIO NOBEL LUC MONTAGNIER SULL...,- Vi farete vaccinare oggi contro il #Covid?\n...,compressed,True,Parsed,192
https://youtu.be/GJrC0xAAXoU,VACCINO: i dubbi più grossi,Ho cercato di fare da tramite tra gli enti per...,compressed,True,Parsed,165


In [3]:
df = pd.read_csv('../Tweets.csv',low_memory=False,usecols=['user_screen_name','urls'])
df.head()

,user_screen_name,urls
0,alessia_smile6,[]
1,GiornaledPuglia,"[{'url': 'https://t.co/bQGJH64Mac', 'expanded_..."
2,TestPerTutti,"[{'url': 'https://t.co/oQZpHUBKrt', 'expanded_..."
3,CapitanHarlok6,[]
4,soteros1,[]


## General info about the video shared

In [4]:
df_yt_summary = df_yt.groupby(['reason']).count()
df_yt_summary

NameError: name 'df_yt' is not defined

In [ ]:
plt.figure(figsize=(20,10))
s = df_yt_summary['occurrency'].sum()
plt.pie(df_yt_summary['occurrency'], labels=df_yt_summary.index, shadow=True, startangle=100,
       autopct=lambda p:f'{p:.2f}% ({p*(s)/100 :,.0f})'.replace(',', '\''),
       explode = [0.25,0.0,0.25,0.0,0.0])
plt.show()

In [ ]:
df_yt = df_yt_complete[np.logical_or(df_yt_complete['reason']=='Parsed' , df_yt_complete['reason']=='Unavailable')]
df_yt

## Tracking user with type youtbe video shared
Done only for unavailbe and parsed video

In [ ]:
df_url = df[df['urls'] != '[]'].copy()

df_url = ut.get_dfYt_line(df_url,df_yt.index)
df_url = df_url[df_url['yt']]

df_url

In [ ]:
df_user_yt = pd.DataFrame(columns=['name','url','type'])

for i in df_url.index:
    user = df.at[i, 'user_screen_name']
    s = df.at[i, 'urls']
    urls = gen_ut.get_string_json(s, 'expanded_url')
    for url in urls:
        if url in df_yt.index:
            df_user_yt.loc[df_user_yt.shape[0]] = [user,url,df_yt.loc[url,'reason']]
            

df_user_yt        

In [ ]:
df_user_yt['isParsed'] = 0
df_user_yt.loc[df_user_yt['type']=='Parsed','isParsed'] = 1
df_user_yt['total'] = 1


df_user_group = df_user_yt.groupby(['name']).sum().sort_values(['total'],ascending=False)
df_user_group

In [ ]:
plt.figure(figsize=(15,10))
plt.barh(df_user_group.head(20).index,df_user_group.head(20).total,label='Total')
plt.barh(df_user_group.head(20).index,df_user_group.head(20).total-df_user_group.head(20).isParsed
        ,label='Unavailbe')
plt.legend()
plt.title('Number of youtube video shared ')